In [4]:
import matplotlib.pyplot as plt
from docx import Document
from docx.shared import Inches, RGBColor
import io

def calcular_concentracion_ion(ppm_compuesto, masa_molar_compuesto, iones_por_molecula, densidad_solucion=1.00):
    """
    Calcula la concentración molar de un ion en solución a partir de ppm del compuesto.
    """
    mg_per_L = ppm_compuesto
    g_per_L = mg_per_L / 1000
    molaridad_compuesto = g_per_L / masa_molar_compuesto
    molaridad_ion = molaridad_compuesto * iones_por_molecula
    return molaridad_ion

def agregar_titulo_negro(doc, texto, nivel=1):
    """
    Agrega un título con formato en negro al documento Word.
    
    Args:
        doc: Objeto Document de python-docx
        texto: Texto del título
        nivel: Nivel de encabezado (1 para principal, 2 para secundario)
    """
    titulo = doc.add_heading(level=nivel)
    run = titulo.add_run(texto)
    run.font.color.rgb = RGBColor(0, 0, 0)  # Negro puro
    return titulo

def generar_informe(ppm_compuesto, masa_molar_compuesto, iones_por_molecula, formula_compuesto, nombre_ion):
    """
    Genera un informe técnico en Word con los resultados del cálculo.
    """
    concentracion = calcular_concentracion_ion(ppm_compuesto, masa_molar_compuesto, iones_por_molecula)
    
    doc = Document()
    
    # 1. Título principal (en negro)
    agregar_titulo_negro(doc, 'Informe Técnico: Cálculo de Concentración Iónica', nivel=1)
    
    # 2. Sección de introducción
    agregar_titulo_negro(doc, 'Introducción', nivel=2)
    doc.add_paragraph(
        f'Este informe presenta el cálculo de la concentración molar del ion {nombre_ion} '
        f'a partir de una solución que contiene {ppm_compuesto} ppm de {formula_compuesto}.'
    )
    
    # 3. Sección de metodología
    agregar_titulo_negro(doc, 'Metodología', nivel=2)
    doc.add_paragraph('Los pasos seguidos para el cálculo son:')
    doc.add_paragraph('1. Convertir ppm a mg/L (asumiendo densidad = 1.00 g/mL)', style='List Bullet')
    doc.add_paragraph('2. Convertir mg/L a g/L', style='List Bullet')
    doc.add_paragraph('3. Convertir g/L a mol/L (molaridad del compuesto)', style='List Bullet')
    doc.add_paragraph(f'4. Multiplicar por {iones_por_molecula} (iones {nombre_ion} por molécula)', style='List Bullet')
    
    # 4. Sección de resultados
    agregar_titulo_negro(doc, 'Resultados', nivel=2)
    doc.add_paragraph(f'Concentración de {formula_compuesto}: {ppm_compuesto} ppm')
    doc.add_paragraph(f'Masa molar: {masa_molar_compuesto} g/mol')
    doc.add_paragraph(f'Número de iones {nombre_ion} por molécula: {iones_por_molecula}')
    doc.add_paragraph(f'Concentración molar del ion {nombre_ion}: {concentracion:.4e} M')
    
    # 5. Crear y agregar gráfica
    agregar_titulo_negro(doc, 'Visualización de Resultados', nivel=2)
    
    fig, ax = plt.subplots(figsize=(8, 4))
    compuestos = [formula_compuesto, nombre_ion]
    concentraciones = [ppm_compuesto, concentracion*1000]
    
    bars = ax.bar(compuestos, concentraciones, color=['blue', 'orange'])
    ax.set_ylabel('Concentración (ppm o mM)', color='black')
    ax.set_title(f'Comparación: {formula_compuesto} vs {nombre_ion}', color='black')
    ax.tick_params(axis='both', colors='black')
    
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.2f}',
                ha='center', va='bottom', color='black')
    
    img_buffer = io.BytesIO()
    plt.savefig(img_buffer, format='png', dpi=300, bbox_inches='tight')
    img_buffer.seek(0)
    plt.close()
    
    doc.add_picture(img_buffer, width=Inches(6))
    
    # 6. Explicación del código
    agregar_titulo_negro(doc, 'Explicación del Código', nivel=2)
    doc.add_paragraph('El código Python implementa los siguientes pasos equivalentes al ejercicio:')
    
    pasos = [
        ("Conversión de ppm a mg/L", "mg_per_L = ppm_compuesto"),
        ("Conversión de mg/L a g/L", "g_per_L = mg_per_L / 1000"),
        ("Cálculo de molaridad del compuesto", "molaridad_compuesto = g_per_L / masa_molar_compuesto"),
        ("Cálculo de molaridad del ion", "molaridad_ion = molaridad_compuesto * iones_por_molecula")
    ]
    
    for paso, codigo in pasos:
        doc.add_paragraph(paso, style='Heading 3')
        doc.add_paragraph(codigo, style='Normal')
    
    doc.save('Informe_Concentracion_Ionica_Dreckell.docx')
    return concentracion

# Ejemplo de uso
if __name__ == "__main__":
    concentracion = generar_informe(
        ppm_compuesto=63.3,
        masa_molar_compuesto=329.3,
        iones_por_molecula=3,
        formula_compuesto="K₃Fe(CN)₆",
        nombre_ion="K⁺"
    )
    print(f"Informe generado con éxito. Concentración de K⁺: {concentracion:.4e} M")

Informe generado con éxito. Concentración de K⁺: 5.7668e-04 M
